In [7]:
import torch  # pytorch backend
from grb.dataset import Dataset
from grb.model.torch import GCN
from grb.utils.trainer import Trainer
from grb.attack.tdgia import TDGIA
import torch  # pytorch backend
from grb.dataset import Dataset
from grb.model.torch import GCN
from grb.utils.normalize import GCNAdjNorm
from grb.defense import AdvTrainer
from torch_geometric.datasets import Planetoid

/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/Projects/GraphUnlearning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# import cora
dataset_name = 'Cora'

# Load the dataset
dataset = Planetoid(root='data', name=dataset_name)

In [ ]:
from grb.dataset import CustomDataset
from torch_geometric.utils import to_scipy_sparse_matrix

In [ ]:
# create custom dataset from planetoid dataset

adj_matrix = to_scipy_sparse_matrix(dataset[0].edge_index)
features = dataset[0].x
labels = dataset[0].y
name = 'cora-custom'
data_dir = 'grb_data/cora'
save = True

In [ ]:
cora_custom = CustomDataset(
    adj=adj_matrix,
    features=features,
    labels=labels,
    name=name,
    data_dir=data_dir,
    save=save,
)

GRB data splitting...
    Average degree of all nodes: 3.8981
    Average degree of 5% nodes with small degree: 1.0000
    Average degree of 5% nodes with large degree: 17.4265
    Average degree of 30% nodes (easy): 1.5690
    Randomly sampled 270 nodes
    Average degree of 30% nodes (medium): 3.0221
    Randomly sampled 270 nodes
    Average degree of 30% nodes (hard): 5.3202
    Randomly sampled 270 nodes
    Number of training/validation nodes: 1624/274
    No duplicate.
    Saved in grb_data/cora.
Custom Dataset 'cora-custom' loaded.
    Number of nodes: 2708
    Number of edges: 5278
    Number of features: 1433
    Number of classes: 7
    Number of train samples: 1624
    Number of val samples: 274
    Number of test samples: 810
    Dataset mode: full
    Feature range [0.0000, 1.0000]


In [ ]:
cora_custom.adj

<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 10556 stored elements in COOrdinate format>

In [ ]:
dataset = Dataset(name='grb-cora', data_dir='data/grb-cora', mode='hard')

Dataset 'grb-cora' loaded.
    Number of nodes: 2708
    Number of edges: 5278
    Number of features: 1433
    Number of classes: 7
    Number of train samples: 1624
    Number of val samples: 274
    Number of test samples: 270
    Dataset mode: hard
    Feature range: [0.0000, 1.0000]


In [ ]:
from grb.model.torch import GCN

In [ ]:
model = GCN(in_features=dataset.num_features,
            out_features=dataset.num_classes,
            hidden_features=[64, 64])

In [ ]:
adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, optimizer=adam, loss=torch.nn.functional.nll_loss)
trainer.train(model=model, n_epoch=200, dropout=0.5, train_mode="inductive")

/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/Projects/GraphUnlearning/.venv/lib/python3.11/site-packages/grb/utils/utils.py:40: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:620.)
  adj_tensor = torch.sparse.FloatTensor(sparse_concat.t(), sparse_data, torch.Size(adj.shape))


Epoch 00000 | Train loss 55.0298 | Train score 0.0844 | Val loss 49.5411 | Val score 0.1277
Epoch 00010 | Best validation score: 0.4234
Model saved in './tmp_2024_05_02_16_21_47/checkpoint.pt'.
Epoch 00010 | Train loss 14.2097 | Train score 0.4181 | Val loss 11.6524 | Val score 0.4234
Epoch 00020 | Best validation score: 0.4270
Model saved in './tmp_2024_05_02_16_21_47/checkpoint.pt'.
Epoch 00020 | Train loss 9.3462 | Train score 0.4021 | Val loss 9.1848 | Val score 0.4270
Epoch 00030 | Best validation score: 0.4708
Model saved in './tmp_2024_05_02_16_21_47/checkpoint.pt'.
Epoch 00030 | Train loss 3.9314 | Train score 0.4606 | Val loss 2.1169 | Val score 0.4708
Epoch 00040 | Train loss 1.7718 | Train score 0.4760 | Val loss 1.9258 | Val score 0.4307
Epoch 00050 | Best validation score: 0.4745
Model saved in './tmp_2024_05_02_16_21_47/checkpoint.pt'.
Epoch 00050 | Train loss 1.4247 | Train score 0.5166 | Val loss 1.5684 | Val score 0.4745
Epoch 00060 | Best validation score: 0.4854
Mode

In [ ]:
from grb.attack.tdgia import TDGIA

# Attack configuration
tdgia = TDGIA(
    lr=0.01,
    n_epoch=10,
    n_inject_max=20,
    n_edge_max=20,
    feat_lim_min=-0.9,
    feat_lim_max=0.9,
    sequential_step=0.2,
)

# covert adj to coo format
adj = dataset.adj.tocoo()

# Apply attack
rst = tdgia.attack(
    model=model,
    adj=adj,
    features=dataset.features,
    target_mask=dataset.test_mask,
    adj_norm_func=GCNAdjNorm,
)
# Get modified adj and features
adj_attack, features_attack = rst

Attacking: Sequential inject 4/20 nodes
Attacking: Epoch 9, Loss: 5.86623, Surrogate test acc: 0.98889
Attacking: Sequential inject 8/20 nodes
Attacking: Epoch 9, Loss: 5.82277, Surrogate test acc: 0.97037
Attacking: Sequential inject 12/20 nodes
Attacking: Epoch 9, Loss: 5.78868, Surrogate test acc: 0.95185
Attacking: Sequential inject 16/20 nodes
Attacking: Epoch 9, Loss: 5.76347, Surrogate test acc: 0.93333
Attacking: Sequential inject 20/20 nodes
Attacking: Epoch 9, Loss: 5.73217, Surrogate test acc: 0.92593


In [ ]:
from grb.utils import evaluate

test_score = evaluate(
    model=model,
    adj=adj_attack,
    features=features_attack,
    labels=dataset.labels,
    mask=dataset.test_mask,
    adj_norm_func=GCNAdjNorm,
)

test_score

ImportError: cannot import name 'evaluate' from 'grb.utils' (/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/Projects/GraphUnlearning/.venv/lib/python3.11/site-packages/grb/utils/__init__.py)